In [1]:
import torch
import torch.nn.functional as F

In [2]:
g = torch.Generator().manual_seed(2147483647)

def get_dataset(names):
    xs, ys = [], []
    for name in names:
        name = '..' + name + '.'
        for chr1, chr2, chr3 in zip(name, name[1:], name[2:]):
            idx1 = chtoi[chr1]
            idx2 = chtoi[chr2]
            idx3 = chtoi[chr3]
            xs.append((idx1, idx2))
            ys.append(idx3)
    return xs, ys

def predict_name(g, get_sample_distribution):
    name = '..'
    idx1, idx2 = 0, 0
    while True:
        sample_distribution = get_sample_distribution(idx1, idx2)
        idx = torch.multinomial(sample_distribution, num_samples=1, replacement=True, generator=g).item()
        idx1 = idx2
        idx2 = idx
        name += itoch[idx]
        if idx == 0:
            break
    return name

def get_loss(probs, y_s):
    return -probs[torch.arange(probs.shape[0]), y_s].log().mean()

## E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?

## Counting

In [3]:
names = open('names.txt', 'r').read().splitlines()

In [4]:
unique_chars = set(''.join(names))
unique_chars = sorted(unique_chars)
chtoi = {ch: i + 1 for i, ch in enumerate(unique_chars)}
chtoi['.'] = 0
itoch = {value:key for key, value in chtoi.items()}
table_counts = torch.zeros(len(chtoi), len(chtoi), len(chtoi), dtype=torch.int32)

In [5]:
xs, ys = get_dataset(names)

In [6]:
for x, y in zip(xs, ys):
    idx1 = x[0]
    idx2 = x[1]
    table_counts[idx1][idx2][y] += 1    

In [7]:
probas = table_counts / table_counts.sum(dim=2, keepdim=True)
g = torch.Generator().manual_seed(2147483647)
for _ in range(20):
    name = predict_name(g, lambda idx1, idx2: probas[idx1][idx2])
    print(name)

..mip.
..axx.
..mereyannyaar.
..knooraen.
..el.
..marviovania.
..odarimalaalexiaganilley.
..helahroni.
..haat.
..affiya.
..isemarrisleemikh.
..bech.
..amilleia.
..trutandenneppalycethon.
..jan.
..kryn.
..yusehanii.
..laymira.
..knoenoah.
..nowynni.


In [8]:
#trigram loss
probs = []
for x, y in zip(xs, ys):
    proba = probas[x].unsqueeze(dim=0)
    probs.append(proba)
probs = torch.cat(probs)
loss = get_loss(probs, ys).item()
print(loss)

2.1857433319091797


## Neural Net

In [9]:
#def map_idxs(idx1, idx2, vocab_len):
#    return len(chtoi) * idx1 + idx2

In [10]:
x_s, y_s = get_dataset(names)
#x_s = [map_idxs(idx1, idx2, len(chtoi) for idx1, idx2 in x_s]
x_s = torch.tensor(x_s)
y_s = torch.tensor(y_s)

In [11]:
#emb_size = len(chtoi)**2
#W = torch.rand(emb_size, len(chtoi), generator=g, requires_grad=True, dtype=torch.float)

In [12]:
# The basic idea is to represent each character as a one-hot vector of length 27, and then concatenate them together
# in this way the final length of input for matrix w will be 27 * 2
W = torch.rand(27*2, len(chtoi), generator=g, requires_grad=True, dtype=torch.float)

In [13]:
def forward(xenc):
    counts = xenc @ W
    logits = counts.exp()
    probs = logits / torch.sum(logits, dim=1, keepdim=True)
    return probs

In [14]:
xenc1 = F.one_hot(x_s[:, 0], num_classes=len(chtoi)).float()
xenc2 = F.one_hot(x_s[:, 1], num_classes=len(chtoi)).float()
xenc = torch.cat((xenc1, xenc2), dim=1)

In [15]:
for i in range(1000):
    probs = forward(xenc)
    loss = get_loss(probs, y_s)
    W.grad = None
    #b.grad = None
    loss.backward()
    W.data -=  20 * W.grad
    if i % 100 == 0:
        print(f'Iteration num - {i}: Training loss: {loss}')

Iteration num - 0: Training loss: 3.4063494205474854
Iteration num - 100: Training loss: 2.383622884750366
Iteration num - 200: Training loss: 2.361172676086426
Iteration num - 300: Training loss: 2.35288667678833
Iteration num - 400: Training loss: 2.348541021347046
Iteration num - 500: Training loss: 2.3458638191223145
Iteration num - 600: Training loss: 2.344055652618408
Iteration num - 700: Training loss: 2.3427603244781494
Iteration num - 800: Training loss: 2.341792345046997
Iteration num - 900: Training loss: 2.3410470485687256


In [16]:
# we didn't achieve the same loss as trigram model, i think this is because of the insufficient complexity of our model. Initially, i tried 
# an architecture with 27**2 input, i.e. for each combination of 27 chars - vector. This approach yielded a loss close to that of the trigram model. 
# However, I find the current architecture more intriguing😅

In [17]:
def nn_get_sample_distribution(idx1, idx2):
    xenc1 = F.one_hot(torch.tensor(idx1).unsqueeze(dim=0), num_classes=len(chtoi)).float()
    xenc2 = F.one_hot(torch.tensor(idx2).unsqueeze(dim=0), num_classes=len(chtoi)).float()
    xenc = torch.cat((xenc1, xenc2), dim=1)
    probs = forward(xenc)
    return probs

In [18]:
g = torch.Generator().manual_seed(2147483647)
for _ in range(20):
    name = predict_name(g, nn_get_sample_distribution)
    print(name)

..mor.
..ays.
..minaymnnyaes.
..konamaloe.
..caonavioy.
..rie.
..oi.
..ondalaek.
..shalekirierielah.
..yshi.
..haat.
..ah.
..kyn.
..ilan.
..lumiogengen.
..be.
..fa.
..jilgila.
..tostandemkrmya.
..zacrevina.


## E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?


In [19]:
# code from lecture
words = open('names.txt', 'r').read().splitlines()

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

# create the dataset
xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)

num_train_elements = int(0.8 * xs.shape[0])
num_val_elements = int(0.9 * xs.shape[0])
train_xs = xs[:num_train_elements]
train_y = ys[:num_train_elements]
val_xs = xs[num_train_elements:num_val_elements]
val_y = ys[num_train_elements:num_val_elements]
test_xs = xs[num_val_elements:]
test_y = ys[num_val_elements:]

num = xs.nelement()
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

number of examples:  228146


In [20]:
# gradient descent
xenc = F.one_hot(train_xs, num_classes=27).float() # input to the network: one-hot encoding
for k in range(100):
  
  # forward pass
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(xenc.shape[0]), train_y].log().mean() + 0.01*(W**2).mean()
  if k % 10 == 0:
      print(f'Iteration num - {k}: Training loss: {loss}')
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()
  
  # update
  W.data += -50 * W.grad

Iteration num - 999: Training loss: 3.777691125869751
Iteration num - 999: Training loss: 2.666555643081665
Iteration num - 999: Training loss: 2.553800582885742
Iteration num - 999: Training loss: 2.51287841796875
Iteration num - 999: Training loss: 2.4926834106445312
Iteration num - 999: Training loss: 2.481161594390869
Iteration num - 999: Training loss: 2.473928213119507
Iteration num - 999: Training loss: 2.469041585922241
Iteration num - 999: Training loss: 2.4655580520629883
Iteration num - 999: Training loss: 2.4629757404327393


In [21]:
# gradient descent
for xs, ys, split_name in [(val_xs, val_y, 'val'), (test_xs, test_y, 'test')]:
    with torch.no_grad():
        xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
          
        # forward pass
        logits = xenc @ W # predict log-counts
        counts = logits.exp() # counts, equivalent to N
        probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
        loss = -probs[torch.arange(xenc.shape[0]), ys].log().mean()
        print(f'loss on {split_name} = {loss.item()}')

loss on val = 2.612541913986206
loss on test = 2.616680145263672


In [22]:
# loss on val and test is almost the same

### Trigram

In [23]:
xs, ys = get_dataset(names)
#x_s = [map_idxs(idx1, idx2, len(chtoi) for idx1, idx2 in x_s]
xs = torch.tensor(xs)
ys = torch.tensor(ys)

num_train_elements = int(0.8 * xs.shape[0])
num_val_elements = int(0.9 * xs.shape[0])
train_xs = xs[:num_train_elements]
train_y = ys[:num_train_elements]
val_xs = xs[num_train_elements:num_val_elements]
val_y = ys[num_train_elements:num_val_elements]
test_xs = xs[num_val_elements:]
test_y = ys[num_val_elements:]

xenc1 = F.one_hot(train_xs[:, 0], num_classes=len(chtoi)).float()
xenc2 = F.one_hot(train_xs[:, 1], num_classes=len(chtoi)).float()
xenc_train = torch.cat((xenc1, xenc2), dim=1)

xenc1 = F.one_hot(val_xs[:, 0], num_classes=len(chtoi)).float()
xenc2 = F.one_hot(val_xs[:, 1], num_classes=len(chtoi)).float()
xenc_val = torch.cat((xenc1, xenc2), dim=1)

xenc1 = F.one_hot(test_xs[:, 0], num_classes=len(chtoi)).float()
xenc2 = F.one_hot(test_xs[:, 1], num_classes=len(chtoi)).float()
xenc_test = torch.cat((xenc1, xenc2), dim=1)

In [24]:
g = torch.Generator().manual_seed(2147483647)
W = torch.rand(27*2, len(chtoi), generator=g, requires_grad=True, dtype=torch.float)

In [25]:
for i in range(200):
    probs = forward(xenc_train)
    loss = get_loss(probs, train_y)
    W.grad = None
    loss.backward()
    if i <= 25:
        lr = 50
    else:
        lr = 25
    W.data -=  lr * W.grad
    if i % 10 == 0:
        print(f'Iteration num - {i}: Training loss: {loss}')

Iteration num - 0: Training loss: 3.4317004680633545
Iteration num - 10: Training loss: 2.4659430980682373
Iteration num - 20: Training loss: 2.422708749771118
Iteration num - 30: Training loss: 2.367277145385742
Iteration num - 40: Training loss: 2.3584823608398438
Iteration num - 50: Training loss: 2.3519515991210938
Iteration num - 60: Training loss: 2.34675669670105
Iteration num - 70: Training loss: 2.3425135612487793
Iteration num - 80: Training loss: 2.3389763832092285
Iteration num - 90: Training loss: 2.3359792232513428
Iteration num - 100: Training loss: 2.333405017852783
Iteration num - 110: Training loss: 2.3311684131622314
Iteration num - 120: Training loss: 2.3292059898376465
Iteration num - 130: Training loss: 2.327470302581787
Iteration num - 140: Training loss: 2.325923204421997
Iteration num - 150: Training loss: 2.324535846710205
Iteration num - 160: Training loss: 2.323284149169922
Iteration num - 170: Training loss: 2.3221497535705566
Iteration num - 180: Training 

In [26]:
# gradient descent
for xs, ys, split_name in [(xenc_val, val_y, 'val'), (xenc_test, test_y, 'test')]:
    with torch.no_grad():
        probs = forward(xs)
        loss = get_loss(probs, ys)
        print(f'loss on {split_name} = {loss.item()}')

loss on val = 2.5141124725341797
loss on test = 2.5235397815704346


In [27]:
# difference a bit larger than expected, but i believe it's still close enough.

## E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?

In [28]:
strenght_to_loss = {}
for strenght in [0.00001, 0.001, 0.01, 0.1, 1.0, 2.0]:
    g = torch.Generator().manual_seed(2147483647)
    W = torch.rand(27*2, len(chtoi), generator=g, requires_grad=True, dtype=torch.float)
    for i in range(200):
        probs = forward(xenc_train)
        loss = get_loss(probs, train_y) + strenght * (W**2).mean()
        W.grad = None
        loss.backward()
        if i <= 25:
            lr = 50
        else:
            lr = 25
        W.data -=  lr * W.grad
    with torch.no_grad():
        val_probs = forward(xenc_val)
        val_loss = get_loss(val_probs, val_y)
        strenght_to_loss[strenght] = val_loss
    print(f'strenght - {strenght}, train_loss - {loss}, val_loss - {val_loss}')

strenght - 1e-05, train_loss - 2.31939959526062, val_loss - 2.5141122341156006
strenght - 0.001, train_loss - 2.3204896450042725, val_loss - 2.514098882675171
strenght - 0.01, train_loss - 2.3298182487487793, val_loss - 2.514075517654419
strenght - 0.1, train_loss - 2.3896865844726562, val_loss - 2.5200507640838623
strenght - 1.0, train_loss - 2.6047399044036865, val_loss - 2.614614963531494
strenght - 2.0, train_loss - 2.7164664268493652, val_loss - 2.6820991039276123


In [29]:
# We can see that as the regularization strength increases, the loss on the training dataset also increases
# The reason is that it becomes harder for the model to overfit because of regularization on the weights
# And for val dataset there exists an optimal value outside of which the loss increases

In [30]:
g = torch.Generator().manual_seed(2147483647)
W = torch.rand(27*2, len(chtoi), generator=g, requires_grad=True, dtype=torch.float)

In [31]:
%%time
for i in range(200):
    probs = forward(xenc_train)
    loss = get_loss(probs, train_y) + 0.01 * (W**2).mean()
    W.grad = None
    loss.backward()
    if i <= 25:
        lr = 50
    else:
        lr = 25
    W.data -=  lr * W.grad

CPU times: user 14.4 s, sys: 4.54 s, total: 18.9 s
Wall time: 18.9 s


In [32]:
with torch.no_grad():
    test_probs = forward(xenc_test)
    test_loss = get_loss(test_probs, test_y)
print(f'test_loss - {test_loss}')

test_loss - 2.523268461227417


## E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?

In [33]:
g = torch.Generator().manual_seed(2147483647)
W = torch.rand(2, 27, len(chtoi), generator=g, requires_grad=True, dtype=torch.float)

In [34]:
%%time
for i in range(200):
    counts = W[0][train_xs[:, 0]] + W[1][train_xs[:, 1]]
    logits = counts.exp()
    probs = logits / torch.sum(logits, dim=1, keepdim=True)
    loss = get_loss(probs, train_y) + 0.01 * (W**2).mean()
    W.grad = None
    loss.backward()
    if i <= 25:
        lr = 50
    else:
        lr = 25
    W.data -=  lr * W.grad
    if i % 10 == 0:
        print(f'Iteration num - {i}: Training loss: {loss}')

Iteration num - 0: Training loss: 3.435056209564209
Iteration num - 10: Training loss: 2.4715957641601562
Iteration num - 20: Training loss: 2.4289772510528564
Iteration num - 30: Training loss: 2.3741769790649414
Iteration num - 40: Training loss: 2.3657381534576416
Iteration num - 50: Training loss: 2.3595263957977295
Iteration num - 60: Training loss: 2.3546226024627686
Iteration num - 70: Training loss: 2.35064697265625
Iteration num - 80: Training loss: 2.3473575115203857
Iteration num - 90: Training loss: 2.344590663909912
Iteration num - 100: Training loss: 2.3422319889068604
Iteration num - 110: Training loss: 2.340198040008545
Iteration num - 120: Training loss: 2.3384268283843994
Iteration num - 130: Training loss: 2.336871385574341
Iteration num - 140: Training loss: 2.335495948791504
Iteration num - 150: Training loss: 2.334270715713501
Iteration num - 160: Training loss: 2.333173990249634
Iteration num - 170: Training loss: 2.3321869373321533
Iteration num - 180: Training 

In [35]:
with torch.no_grad():
    counts = W[0][test_xs[:, 0]] + W[1][test_xs[:, 1]]
    logits = counts.exp()
    test_probs = logits / torch.sum(logits, dim=1, keepdim=True)
    test_loss = get_loss(test_probs, test_y)
print(f'test_loss - {test_loss}')

test_loss - 2.523268699645996


## E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?

In [36]:
g = torch.Generator().manual_seed(2147483647)
W = torch.rand(2, 27, len(chtoi), generator=g, requires_grad=True, dtype=torch.float)

In [37]:
%%time
for i in range(200):
    counts = W[0][train_xs[:, 0]] + W[1][train_xs[:, 1]]
    loss = F.cross_entropy(input=counts, target=train_y) + 0.01 * (W**2).mean()
    W.grad = None
    loss.backward()
    if i <= 25:
        lr = 50
    else:
        lr = 25
    W.data -=  lr * W.grad
    if i % 10 == 0:
        print(f'Iteration num - {i}: Training loss: {loss}')

Iteration num - 0: Training loss: 3.435056686401367
Iteration num - 10: Training loss: 2.4715957641601562
Iteration num - 20: Training loss: 2.4289772510528564
Iteration num - 30: Training loss: 2.3741767406463623
Iteration num - 40: Training loss: 2.3657381534576416
Iteration num - 50: Training loss: 2.3595263957977295
Iteration num - 60: Training loss: 2.3546223640441895
Iteration num - 70: Training loss: 2.35064697265625
Iteration num - 80: Training loss: 2.3473572731018066
Iteration num - 90: Training loss: 2.344590663909912
Iteration num - 100: Training loss: 2.3422319889068604
Iteration num - 110: Training loss: 2.340198040008545
Iteration num - 120: Training loss: 2.3384268283843994
Iteration num - 130: Training loss: 2.3368711471557617
Iteration num - 140: Training loss: 2.335495710372925
Iteration num - 150: Training loss: 2.334270477294922
Iteration num - 160: Training loss: 2.3331737518310547
Iteration num - 170: Training loss: 2.3321871757507324
Iteration num - 180: Trainin

In [38]:
with torch.no_grad():
    counts = W[0][test_xs[:, 0]] + W[1][test_xs[:, 1]]
    test_loss = F.cross_entropy(input=counts, target=test_y)
print(f'test_loss - {test_loss}')

test_loss - 2.523268699645996


In [39]:
# We'd prefer to use  F.cross_entropy because of effectiveness forward and backward passes, as well as its numerical stability